In [1]:
import os.path
import numpy as np
import pandas as pd
from scipy.fft import fft

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.animation import FuncAnimation
%matplotlib notebook

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#display(HTML(f"<style>.container {{ height: {400}px !important; }}</style>"))

In [2]:
file_folder = '/home/anna/annaCode/UiO/MEK4350/notebooks/Data_undervisning200923'
date_format = '%m/%d/%Y %H:%M:%S.%f'

df_list = []
for i in range(1,5):
    file_path =  os.path.join(file_folder, f"proberun{i}_tett.csv")
    df = pd.read_csv(file_path, names = ['date_string', 'probe_1_raw', 'probe_2_raw', 'probe_3_raw', 'probe_4_raw', 'sensors'])
    df['date_time'] = pd.to_datetime(df['date_string'], format=date_format)
    df['elapsed_secs'] =  (df['date_time'] - df.at[0, 'date_time']).dt.total_seconds()
    #calculate amplitudes
    df['probe_1'] = df['probe_1_raw'].mean() - df['probe_1_raw']
    df['probe_2'] = df['probe_2_raw'].mean() - df['probe_2_raw']
    df['probe_3'] = df['probe_3_raw'].mean() - df['probe_3_raw']
    df['probe_4'] = df['probe_4_raw'].mean() - df['probe_4_raw']

    df_list.append(df)
    
df = df_list[0]
df.head(2)

,date_string,probe_1_raw,probe_2_raw,probe_3_raw,probe_4_raw,sensors,date_time,elapsed_secs,probe_1,probe_2,probe_3,probe_4
0,09/20/2023 08:42:24.832,0.10401,0.10358,0.10444,0.10367,343.87,2023-09-20 08:42:24.832,0.000,-0.000255,-0.000142,-0.000671,0.000152
1,09/20/2023 08:42:24.836,0.10410,0.10358,0.10444,0.10367,343.87,2023-09-20 08:42:24.836,0.004,-0.000345,-0.000142,-0.000671,0.000152


In [3]:
sample_no = df.shape[0]
sampling_rate = sample_no/df.elapsed_secs.iloc[-1]

In [4]:
frequencies_list = []
magnitude_list = []
growth_list = []

for i in range (0,4):
    frequencies = np.fft.fftfreq(sample_no, d=1/sampling_rate)
    magnitude_spectrum = np.abs(fft(df[f'probe_{i+1}'].to_numpy()))
    size_diff = frequencies.shape[0] - magnitude_spectrum.shape[0]
    if size_diff > 0:
        frequencies = frequencies[:-size_diff]  
    frequencies_list.append(frequencies)
    magnitude_list.append(magnitude_spectrum)
    indices = np.where(magnitude_spectrum > 50)
    growth_list.append(magnitude_spectrum[indices][0])

In [5]:
growth_list

[164.87036326957883, 153.28411430026466, 118.40095643184628, 70.85420859995241]

In [6]:
np.log(abs(growth_list[1]/growth_list[0]))

-0.07286633234885752

In [7]:
real_distances = np.array([6.98, 8.155, 12.325, 16.995])
growth_rates = []
for i in range (0,3):
    growth_rate = 1 / (real_distances[i+1] - real_distances[i])*np.log(growth_list[i+1]/growth_list[i])
    growth_rates.append(growth_rate)

In [8]:
growth_rates

[-0.062013899871368156, -0.06192238733876461, -0.1099469881080432]